# Generate Layout

The first step is to create a layout and test manifest for your project, so that you can relate measurements and analyses back to your source layout.

You can generate the layout with any layout tool. In this notebook you can use gdsfactory.

## Generate layout

You can download this [generate_layout.py](./generate_layout.py) file

We will sweep the width of a 20um long sheet to measure sheet resistance.

In [ ]:
from generate_layout import top

c = top()
c.write_gds("test_chip.gds")
c.plot()

## Generate design manifest

In your sample GDS, you have the device settings annotated on the GDS.
Here you can read them from the GDS and write a design manifest, which we can use to associate measurement data with the devices on the GDS. However, you can use any method you prefer to generate your test manifest.

In [ ]:
import csv

sweep = c.kcl["resistance"]
csvpath = "design_manifest.csv"

analysis = "[iv_resistance]"
analysis_parameters = "[{}]"
length = 20

with open(csvpath, "w") as f:
    writer = csv.writer(f)
    writer.writerow(
        [
            "cell",
            "x",
            "y",
            "width_um",
            "length_um",
            "analysis",
            "analysis_parameters",
        ]
    )
    iterator = sweep.begin_instances_rec()
    iterator.targets = "resistance*"
    while not iterator.at_end():
        _c = c.kcl[iterator.inst_cell().cell_index()]
        name = _c.name
        if "resistance_" in name:
            _disp = (iterator.trans() * iterator.inst_trans()).disp
            params = name.split("_")  # we extract the width from the device name
            writer.writerow(
                [
                    name,
                    _disp.x,
                    _disp.y,
                    float(params[-1][1:]),
                    length,
                    analysis,
                    analysis_parameters,
                ]
            )
        iterator.next()

In [ ]:
params

You can take a look a the contents of the device manifest you created.

In [ ]:
import pandas as pd

df = pd.read_csv(csvpath)
df